### Downloading datasets:

1. download image sequences from: http://www.cvlibs.net/download.php?file=data_odometry_color.zip
2. download ground truth labels from: http://www.cvlibs.net/download.php?file=data_odometry_poses.zip
3. download flow ground truth from: https://drive.google.com/file/d/1U9FajKofxIIpT4lRR0oAYgV1plxzRo4f/view?usp=sharing
3. Unzip the downloaded contents in the 'path_to_dataset' folder

In [1]:
train= True
debug=False
epochs=35

In [1]:
exp_name='exp_name/'
data_root='path_to_data_dirr\\'
rep_root='path_to_working_dirr\\'
notebook_name='prf_training_notebook_original.ipynb'
load_random_vars=False
load_init_wts=False
random_vars_dir=rep_root+'PRF\\logs\\load_vars\\'
log_dir=rep_root+'PRF\\logs\\'
start_epoch=1
#wt_path=rep_root+'Flow-Net\\logs\\exp9-lr-0.0001-ns-1.0\\exp9-lr-0.0001-ns-1.0best_wts.hdf5'
wt_path=''#rep_root+'Flow-Net\\logs\\exp6-lr-0.0001-ns-1.0\\exp6-lr-0.0001-ns-1.0best_wts.hdf5'


pretrain_exp_name=None#'exp-M49\\'#-layer_not_trained_0-lr-0.0005-odom_wt/'
start_epoch+=0

if pretrain_exp_name == None :
    pre_trained_wts_path=None#log_dir+pretrain_exp_name+str(start_epoch-1)+".hdf5"#log_dir+exp_name+"10.hdf5"
    pre_trained_opt_path=None#log_dir+pretrain_exp_name+str(start_epoch-1)+".pkl"
    pre_trained_log_path=None#log_dir+pretrain_exp_name+'loss_epoch.csv'
else :
    pre_trained_wts_path=log_dir+pretrain_exp_name+str(start_epoch-1)+".hdf5"
    pre_trained_opt_path=log_dir+pretrain_exp_name+str(start_epoch-1)+".pkl"
    pre_trained_log_path=log_dir+pretrain_exp_name+'loss_epoch.csv'

#best_wts_path=log_dir+exp_name+'best_model'+'.h5'
best_wts_path=log_dir+exp_name+'best_model_rot'+'.h5'
#best_wts_path=log_dir+exp_name+'best_model_trans'+'.h5'

img_root=data_root+'data_odometry_color\\dataset\\sequences\\'
label_root=data_root+'data_odometry_poses\\dataset\\poses\\'
seed=1
drop_seed=seed

In [3]:
## from __future__ import print_function
import os
os.environ['PYTHONHASHSEED']=str(0)
import random
random.seed(seed)
import numpy as np
np.random.seed(seed)
import tensorflow as tf
os.environ['TF_DETERMINISTIC_OPS'] = '1'
tf.random.set_seed(seed)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense ,Input,concatenate ,Conv2D,Conv2DTranspose,\
MaxPooling2D,AveragePooling2D, LSTM ,Reshape, TimeDistributed,ReLU, LeakyReLU, Dropout, BatchNormalization
from tensorflow.python.keras.utils.vis_utils import plot_model
from tensorflow.keras.optimizers import Adam,Adagrad
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K


import glob
import math
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import pickle
import h5py
from collections import deque
from copy import copy,deepcopy
from tqdm import tqdm,trange
import time
import zlib

image_w= 320# 1280
image_h= 128#384
time_step=2
overlap=1
if train ==False:
    overlap=time_step-1
start_points=[0]#,2,4]
no_of_pose=6

#all_files=[0,1,2,3,4,5,6,7,8,9,10]
train_files=[0,1,2,3,6,8,9]
val_files = [5,7,10]#7,10]
test_files= [4,5,7,10]

#learning rate scheduling parameters
batch_size=16
learning_rate=0.0005
_lr_decay_enable=0
_lr_decay_ratio = 2
_lr_decay_epoch=200
_lr_min=0.0001
_grad_clip=0.000002#1e-6

scale=2
#label_scale='std_scl'
label_scale='diff_div'
#label_scale='diff_scl'
_img_norm=False
_img_offset=0.5
_keep_yaw_only=False
_flow_scale=2
_flow_format='hsv'
_flow_loss='rmsle'
_flow_chanel=2
_flow_decompress=False

_disp_or_seg='disp'
_pose_loss='mae'
_disp_scale=2
_disp_loss=_flow_loss
_disp_chanel=1

time_dis=False
repeat_data=False

_grad_lr=0.00000 
_grad_alpha=1.5
_init_loss_wts=np.array([1,10,0.1,0],dtype=np.float32)
_mtl_main_task_index=1
_mtl_aux_grad_algo='v4'

_mtl_cosine_use_fact=0.0
_mtl_cos_alpha=1.0
_mtl_cos_center=1.0
_mtl_cosine_avg='eqn_1' 

_mtl_use_loss_slope='none'
_mtl_loss_center=1.0
_mtl_loss_slope_center=1.0
_mtl_loss_slope_alpha=1.0



_mtl_grad_sign_fact=0.1
_mtl_grad_diff='square'

_mtl_get_variance_from='calc'
_mtl_grad_var_calc_every=1
_mtl_update_grad_var=1
_mtl_grad_var_alpha=1.0
_mtl_div_with_rolling_mean=False
_mtl_use_std_ratio=True
_mtl_use_factors='default' #'ratio_only' or 'coef_only' or 'default'
_mtl_beta_norm_method='none'
_mtl_beta_use_fact=1.0
_mtl_take_log_prob=False
_mtl_use_cosine_v4=False
_mtl_use_mode_v4='mode-1'
_mtl_mode_2_alpha_v4=0.5

#loss weights decay parameters
_disp_weight_decay_enable=0
_disp_weight_decay_ratio = 100
_disp_weight_decay_epoch=10
_disp_weight_min=0.0001

_flow_weight_decay_enable=0
_flow_weight_decay_ratio = 10
_flow_weight_decay_epoch=10
_flow_weight_min=0.0001

#curr_optimizer=Adagrad(lr=learning_rate)#,clipvalue=_grad_clip)
#opt=Adam(lr=learning_rate)#, beta_1=0.9, beta_2=0.999, amsgrad=False)
curr_optimizer=Adam(lr=learning_rate)
#grad_optimizer=Adam(lr=_grad_lr)
class Net_Params():
    def __init__(self,_scale):
        self.N=64//_scale
        N=self.N
        self._num_shared_layer=9
        
        self._kernels        =[7, 5,  5,  3,  3,  3,  3,  3,   3,    3]
        self._filters        =[N,2*N,4*N,4*N,8*N,8*N,8*N,8*N,16*N,16*N]
        self._strides        =[2, 2,  2,  1,  2,  1,  2,  1,   2,   1]
        self._block          =[1, 2,  3,  3,  4,  4,  5,  5,   6,   6]
        self._nconv          =[1, 1,  1,  2,  1,  2,  1,  2,   1,   2]
        self._act            =[1, 1,  1,  1,  1,  1,  1,  1,   1,   1]
        self._batch_norm     =[1, 1,  1,  1,  1,  1,  1,  1,   1,   1]
        self._drop_out       =[0, 0,  0,  0,  0,  0,  0,  0,   0,   0] 
        self._act[self._num_shared_layer-1]=0
        self._drop_out[self._num_shared_layer-1]=0.0
        
        
        
        
        self._num_trans_layers=2
        self._trans_layer_nodes  = [512,512,512]
        self._trans_act=[1,1,0]
        self._trans_drop_out=[0,0,0]
        
        self._num_rot_layers=2
        self._rot_layer_nodes = [512,512,512]
        self._rot_act=[1,1,0]
        self._rot_drop_out=[0,0,0]
        
        self.drop_seed       =seed
        self._leaky_relu     = True
        self._relu           = False
        self._kernel_initializer='he_normal'
        self._kernel_regularizer=None
        
        self._deconv_filters=[8*N,4*N,2*N,1*N]
        self._deconv_act =[1,1,1,1]
        self._flow_chanel =_flow_chanel
        self._disp_chanel =_disp_chanel
        
        
        
_net_par=Net_Params(scale)

grad_clip_layers= {
    'block_1_conv_1/kernel:0':_grad_clip,
    'block_2_conv_1/kernel:0':_grad_clip,
    'block_3_conv_1/kernel:0':_grad_clip,
    'block_3_conv_2/kernel:0':_grad_clip,
    'block_4_conv_1/kernel:0':_grad_clip,
    'block_4_conv_2/kernel:0':_grad_clip,
    'block_5_conv_1/kernel:0':_grad_clip,
    'block_5_conv_2/kernel:0':_grad_clip,
    'block_6_conv_1/kernel:0':_grad_clip,
}



In [4]:
_net_par.__dict__

{'N': 32,
 '_num_shared_layer': 9,
 '_kernels': [7, 5, 5, 3, 3, 3, 3, 3, 3, 3],
 '_filters': [32, 64, 128, 128, 256, 256, 256, 256, 512, 512],
 '_strides': [2, 2, 2, 1, 2, 1, 2, 1, 2, 1],
 '_block': [1, 2, 3, 3, 4, 4, 5, 5, 6, 6],
 '_nconv': [1, 1, 1, 2, 1, 2, 1, 2, 1, 2],
 '_act': [1, 1, 1, 1, 1, 1, 1, 1, 0, 1],
 '_batch_norm': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 '_drop_out': [0, 0, 0, 0, 0, 0, 0, 0, 0.0, 0],
 '_num_trans_layers': 2,
 '_trans_layer_nodes': [512, 512, 512],
 '_trans_act': [1, 1, 0],
 '_trans_drop_out': [0, 0, 0],
 '_num_rot_layers': 2,
 '_rot_layer_nodes': [512, 512, 512],
 '_rot_act': [1, 1, 0],
 '_rot_drop_out': [0, 0, 0],
 'drop_seed': 1,
 '_leaky_relu': True,
 '_relu': False,
 '_kernel_initializer': 'he_normal',
 '_kernel_regularizer': None,
 '_deconv_filters': [256, 128, 64, 32],
 '_deconv_act': [1, 1, 1, 1],
 '_flow_chanel': 2,
 '_disp_chanel': 1}

In [5]:
param_str=''
param_str+='exp_name : '+str(exp_name)+'\n'
param_str+='notebook_name : '+str(notebook_name)+'\n'
param_str+='load_random_vars : '+str(load_random_vars)+'\n'
param_str+='load_init_wts : '+str(load_init_wts)+'\n'
param_str+='height : '+str(image_h)+'\n'
param_str+='width : '+str(image_w)+'\n'
param_str+='time_step : '+str(time_step)+'\n'
param_str+='overlap : '+str(overlap)+'\n'
param_str+='start_point : '+str(start_points)+'\n'
param_str+='img_offset : '+str(_img_offset)+'\n'
param_str+='img_norm : '+str(_img_norm)+'\n'
param_str+='keep_yaw_only : '+str(_keep_yaw_only)+'\n'
param_str+='label scale : '+str(label_scale)+'\n'
param_str+='repeat_data : '+str(repeat_data)+'\n'
param_str+='pose loss : '+str(_pose_loss)+'\n'
param_str+='learning_rate : '+str(learning_rate)+'\n'
param_str+='lr_decay_enable : '+str(_lr_decay_enable)+'\n'
param_str+='lr_decay_ratio : '+str(_lr_decay_ratio)+'\n'
param_str+='lr_decay_epoch : '+str(_lr_decay_epoch)+'\n'
param_str+='lr_min : '+str(_lr_min)+'\n'
param_str+='grad_lr : '+str(_grad_lr)+'\n'
param_str+='grad_alpha : '+str(_grad_alpha)+'\n'
param_str+='grad_clip : '+str(_grad_clip)+'\n'
param_str+='epochs : '+str(epochs)+'\n'
param_str+='batch_size : '+str(batch_size)+'\n'
param_str+='net_scale : '+str(scale)+'\n'
param_str+='init_loss_wts : '+str(_init_loss_wts)+'\n'

param_str+='disp_or_seg : '+str(_disp_or_seg)+'\n'
param_str+='disp_weight_decay_enable : '+str(_disp_weight_decay_enable)+'\n'
param_str+='disp_weight_decay_ratio : '+str(_disp_weight_decay_ratio)+'\n'
param_str+='disp_weight_decay_epoch : '+str(_disp_weight_decay_epoch)+'\n'
param_str+='disp_weight_min : '+str(_disp_weight_min)+'\n'

param_str+='flow_weight_decay_enable : '+str(_flow_weight_decay_enable)+'\n'
param_str+='flow_weight_decay_ratio : '+str(_flow_weight_decay_ratio)+'\n'
param_str+='flow_weight_decay_epoch : '+str(_flow_weight_decay_epoch)+'\n'
param_str+='flow_weight_min : '+str(_flow_weight_min)+'\n'

param_str+='of_scale : '+str(_flow_scale)+'\n'
param_str+='flow_format : '+str(_flow_format)+'\n'
param_str+='flow_loss : '+str(_flow_loss)+'\n'
param_str+='disp_scale : '+str(_disp_scale)+'\n'
param_str+='disp_loss : '+str(_disp_loss)+'\n'

param_str+='mtl_main_task_index : '+str(_mtl_main_task_index)+'\n'
param_str+='mtl_aux_grad_algo : '+str(_mtl_aux_grad_algo)+'\n'

param_str+='mtl_cosine_use_fact : ' +str(_mtl_cosine_use_fact)+'\n'
param_str+='mtl_cos_alpha : ' +str(_mtl_cos_alpha)+'\n'
param_str+='mtl_cos_center : ' +str(_mtl_cos_center)+'\n'
param_str+='mtl_cosine_avg : ' +str(_mtl_cosine_avg)+'\n'

param_str+='mtl_use_loss_slope : ' +str(_mtl_use_loss_slope)+'\n'
param_str+='mtl_loss_slope_center : '+str(_mtl_loss_slope_center)+'\n'
param_str+='mtl_loss_high_offset : '+str(_mtl_loss_center)+'\n'
param_str+='mtl_loss_slope_alpha : '+str(_mtl_loss_slope_alpha)+'\n'

param_str+='mtl_grad_sign_fact : ' +str(_mtl_grad_sign_fact)+'\n'
param_str+='mtl_grad_diff : ' +str(_mtl_grad_diff)+'\n'

param_str+='mtl_get_variance_from : ' +str(_mtl_get_variance_from)+'\n'
param_str+='mtl_grad_var_calc_every : ' +str(_mtl_grad_var_calc_every)+'\n'
param_str+='mtl_update_grad_var : ' +str(_mtl_update_grad_var)+'\n'
param_str+='mtl_grad_var_alpha : ' +str(_mtl_grad_var_alpha)+'\n'
param_str+='mtl_div_with_rolling_mean : ' +str(_mtl_div_with_rolling_mean)+'\n'

param_str+='mtl_use_std_ratio : ' +str(_mtl_use_std_ratio)+'\n'
param_str+='mtl_use_factors : ' +str(_mtl_use_factors)+'\n'
param_str+='mtl_beta_norm_method : ' +str(_mtl_beta_norm_method)+'\n'
param_str+='mtl_beta_use_fact : ' +str(_mtl_beta_use_fact)+'\n'
param_str+='mtl_prob_scale_fact : ' +str(_mtl_take_log_prob)+'\n'


param_str+='mtl_use_cosine_v4 : ' +str(_mtl_use_cosine_v4)+'\n'
param_str+='mtl_use_mode_v4 : ' +str(_mtl_use_mode_v4)+'\n'
param_str+='mtl_mode_2_alpha_v4 : ' +str(_mtl_mode_2_alpha_v4)+'\n'

for kk,val in _net_par.__dict__.items():
    param_str+=str(kk)+' : '
    param_str+=str(val)+'\n'
for kk,val in grad_clip_layers.items():
    param_str+=str('clip_value - ')+str(kk)+' : '
    param_str+=str(val)+'\n'
#param_str+='loss weights : '+str(loss_weights)+'\n'
param_str+='flow_wt_path : '+str(wt_path)+'\n'
param_str+='seed : '+str(seed)+'\n'
if train==True:
    if not  os.path.exists(log_dir+exp_name): os.mkdir(log_dir+exp_name)
    f=open(log_dir+exp_name+'params.txt','w+')
    f.write(param_str)
    f.close()

In [6]:
_grad_log_epoch=1
_calc_grad_log_epoch=1
_all_wts_log_epoch=1
_save_opts=False
_grad_data_len=100#batch
_train_flow=True
_train_disp=False

grad_log_layers= [
    'block_1_conv_1/kernel:0',
    'block_1_conv_1/bias:0',
    'block_2_conv_1/kernel:0',
    'block_2_conv_1/bias:0',
    'block_3_conv_1/kernel:0',
    'block_3_conv_1/bias:0',
    'block_3_conv_2/kernel:0',
    'block_3_conv_2/bias:0',
    'block_4_conv_1/kernel:0',
    'block_4_conv_1/bias:0',
    'block_4_conv_2/kernel:0',
    'block_4_conv_2/bias:0',
    'block_5_conv_1/kernel:0',
    'block_5_conv_1/bias:0',
    'block_5_conv_2/kernel:0',
    'block_5_conv_2/bias:0',
    'block_6_conv_1/kernel:0',
    'block_6_conv_1/bias:0',
    'dense_1.1/kernel:0',
    'dense_1.1/bias:0',
    'dense_1.2/kernel:0',
    'dense_1.2/bias:0',
    'dense_2.1/kernel:0',
    'dense_2.1/bias:0',
    'dense_2.2/kernel:0',
    'dense_2.2/bias:0'
    ]
    
grad_norm_layers= [
    'block_6_conv_1/kernel:0',
    ]
    
    


### Saving copy of notebook

In [ ]:
if train==True:
    curr_time=time.time()
    print('System time : ',curr_time)
    %autosave 1
    time.sleep(3)
    with open(notebook_name,'r') as file:
        notebook_data=file.read()
    with open(log_dir+exp_name+notebook_name+'_log_'+str(curr_time)+'.ipynb','w') as file:
        file.write(notebook_data)
    %autosave 120

System time :  1607006544.2924528


Autosaving every 1 seconds


### Euler angle to transformation matrix conversion and vice varsa

In [ ]:
_EPS = np.finfo(float).eps * 4.0

# axis sequences for Euler angles
_NEXT_AXIS = [1, 2, 0, 1]

# map axes strings to/from tuples of inner axis, parity, repetition, frame
_AXES2TUPLE = {
    'sxyz': (0, 0, 0, 0), 'sxyx': (0, 0, 1, 0), 'sxzy': (0, 1, 0, 0),
    'sxzx': (0, 1, 1, 0), 'syzx': (1, 0, 0, 0), 'syzy': (1, 0, 1, 0),
    'syxz': (1, 1, 0, 0), 'syxy': (1, 1, 1, 0), 'szxy': (2, 0, 0, 0),
    'szxz': (2, 0, 1, 0), 'szyx': (2, 1, 0, 0), 'szyz': (2, 1, 1, 0),
    'rzyx': (0, 0, 0, 1), 'rxyx': (0, 0, 1, 1), 'ryzx': (0, 1, 0, 1),
    'rxzx': (0, 1, 1, 1), 'rxzy': (1, 0, 0, 1), 'ryzy': (1, 0, 1, 1),
    'rzxy': (1, 1, 0, 1), 'ryxy': (1, 1, 1, 1), 'ryxz': (2, 0, 0, 1),
    'rzxz': (2, 0, 1, 1), 'rxyz': (2, 1, 0, 1), 'rzyz': (2, 1, 1, 1)}

_TUPLE2AXES = dict((v, k) for k, v in _AXES2TUPLE.items())

def euler_to_matrix(ai, aj, ak, axes='syxz'):
    try:
        firstaxis, parity, repetition, frame = _AXES2TUPLE[axes]
    except (AttributeError, KeyError):
        _TUPLE2AXES[axes]  # validation
        firstaxis, parity, repetition, frame = axes

    i = firstaxis
    j = _NEXT_AXIS[i+parity]
    k = _NEXT_AXIS[i-parity+1]

    if frame:
        ai, ak = ak, ai
    if parity:
        ai, aj, ak = -ai, -aj, -ak

    si, sj, sk = math.sin(ai), math.sin(aj), math.sin(ak)
    ci, cj, ck = math.cos(ai), math.cos(aj), math.cos(ak)
    cc, cs = ci*ck, ci*sk
    sc, ss = si*ck, si*sk

    M = np.identity(3)
    if repetition:
        M[i, i] = cj
        M[i, j] = sj*si
        M[i, k] = sj*ci
        M[j, i] = sj*sk
        M[j, j] = -cj*ss+cc
        M[j, k] = -cj*cs-sc
        M[k, i] = -sj*ck
        M[k, j] = cj*sc+cs
        M[k, k] = cj*cc-ss
    else:
        M[i, i] = cj*ck
        M[i, j] = sj*sc-cs
        M[i, k] = sj*cc+ss
        M[j, i] = cj*sk
        M[j, j] = sj*ss+cc
        M[j, k] = sj*cs-sc
        M[k, i] = -sj
        M[k, j] = cj*si
        M[k, k] = cj*ci
    return M

def matrix_to_euler(matrix, axes='syxz'):
    try:
        firstaxis, parity, repetition, frame = _AXES2TUPLE[axes.lower()]
    except (AttributeError, KeyError):
        _TUPLE2AXES[axes]  # validation
        firstaxis, parity, repetition, frame = axes

    i = firstaxis
    j = _NEXT_AXIS[i+parity]
    k = _NEXT_AXIS[i-parity+1]

    M = np.array(matrix, dtype=np.float64, copy=False)[:3, :3]
    if repetition:
        sy = math.sqrt(M[i, j]*M[i, j] + M[i, k]*M[i, k])
        if sy > _EPS:
            ax = math.atan2( M[i, j],  M[i, k])
            ay = math.atan2( sy,       M[i, i])
            az = math.atan2( M[j, i], -M[k, i])
        else:
            ax = math.atan2(-M[j, k],  M[j, j])
            ay = math.atan2( sy,       M[i, i])
            az = 0.0
    else:
        cy = math.sqrt(M[i, i]*M[i, i] + M[j, i]*M[j, i])
        if cy > _EPS:
            ax = math.atan2( M[k, j],  M[k, k])
            ay = math.atan2(-M[k, i],  cy)
            az = math.atan2( M[j, i],  M[i, i])
        else:
            ax = math.atan2(-M[j, k],  M[j, j])
            ay = math.atan2(-M[k, i],  cy)
            az = 0.0

    if parity:
        ax, ay, az = -ax, -ay, -az
    if frame:
        ax, az = az, ax
    return ax, ay, az

def normalize_angle_delta(angle):
    if(angle > np.pi):
        angle = angle - 2 * np.pi
    elif(angle < -np.pi):
        angle = 2 * np.pi + angle
    return angle


def get_ground_6d_poses(p):
    """ For 6dof pose representaion """
    pos = np.array([p[3],p[7], p[11]])
    R = np.array([[p[0], p[1], p[2]], [p[4], p[5], p[6]], [p[8], p[9], p[10]]])
    #angles = matrix_to_euler_2(R)
    angles = matrix_to_euler(R)
    #angles = rotationMatrixToEulerAngles(R)
    return [np.concatenate((pos, angles)),R.reshape(3,3)]

### kitti ground truth matrix (3x4) to relative pose sequence conversion

In [ ]:
def integrate_path(dps):
    N=len(dps)
    paths=np.zeros((N,no_of_pose),dtype=np.float64)
    _dps=[dp[0] for dp in dps]
    for i,dp in enumerate(_dps):
        if i>0: paths[i]=paths[i-1]+(dp)
    return paths

def get_labels_rel_poses(pose_seqs):
    if time_step>2 :rel_pose_seqs=np.zeros((pose_seqs.shape[0],pose_seqs.shape[1]-1,no_of_pose),dtype=np.float64)
    else : rel_pose_seqs=np.zeros((pose_seqs.shape[0],no_of_pose),dtype=np.float64)
    for i,seq in enumerate(pose_seqs):
        rel_poses=np.zeros((time_step,no_of_pose),dtype=np.float64)
        for j,points in enumerate(seq):
                pose,R=get_ground_6d_poses(points)
                if j==0:
                    R_offset=R.T
                    trans_offset=pose
                    rel_poses[j,:]=pose
                else:
                    pose-=trans_offset
                    pose[:3]=R_offset.dot(pose[:3])
                    rel_poses[j,:]=pose
        rel_poses[2:]=rel_poses[2:]-rel_poses[1:-1]
        for j,pose in enumerate(rel_poses):
            rel_poses[j,3:]=[normalize_angle_delta(ang) for ang in pose[3:]] 
            if _keep_yaw_only==True: rel_poses[j,3:]=[rel_poses[j,3],0,0]
        if time_step>2 :rel_pose_seqs[i,:,:]=rel_poses[1:,:].copy()   
        else :rel_pose_seqs[i,:]=rel_poses[1,:].copy()   
    return rel_pose_seqs

def get_labels_seq(label_root1,_files,_overlap=1):
    cords=[]
    poses=[]
    indx={}
    label_root2='*txt'
    #if test==True: overlap=time_step-1
    for start_point in start_points:
        for i in _files:
            for path in glob.glob(label_root1+str(i).zfill(2)+label_root2):
                    f=open(path)
                    start=len(poses)
                    pose_queue=deque(maxlen=time_step)
                    _cords=[]
                    for j,line in enumerate(list(f)[start_point:-1]):
                        points=np.array([float(point) for point in line.split()])
                        _cords.append(points)
                        pose_queue.append(points)
                        if len(pose_queue)==time_step:
                                poses.append(list(pose_queue))
                                pose_queue=deque(list(pose_queue)[-_overlap:],maxlen=time_step)  
                    #poses=poses[:-1]
                    end=len(poses)
                    indx[i]=[start,end]
                    f.close()
                    cords+=_cords
    cords=np.array(cords)
    poses=np.array(poses)
    return cords,poses,indx


In [ ]:
train_cords,abs_train_label,train_indx=get_labels_seq(label_root,train_files,_overlap=overlap)
val_cords,abs_val_label,val_indx=get_labels_seq(label_root,val_files,_overlap=overlap)
test_cords,abs_test_label,test_indx=get_labels_seq(label_root,test_files,_overlap=(time_step-1))


train_label=get_labels_rel_poses(abs_train_label)
val_label=get_labels_rel_poses(abs_val_label)
test_label=get_labels_rel_poses(abs_test_label)

In [ ]:
def get_label_abs_pose_tds(__label):
    _label=__label.copy()
    cords=np.zeros((_label.shape[0],no_of_pose))
    for i,label in enumerate(_label):
        if i==0 :cords[i]+=label[-1]
        else:
            #rot_mat=euler_to_matrix_2([cords[i-1,3],0,0])
            rot_mat=euler_to_matrix(cords[i-1,3],0,0)
            last_pose=label[-1]
            last_pose[:3]=rot_mat.dot(last_pose[:3])
            last_pose+=cords[i-1]
            last_pose[3] = (last_pose[3] + np.pi) % (2 * np.pi) - np.pi
            cords[i]=last_pose
    return cords.copy()
def get_label_abs_pose(__label):
    _label=__label.copy()
    cords=np.zeros((_label.shape[0],no_of_pose))
    for i,label in enumerate(_label):
        if i==0 :cords[i]+=label
        else:
            #rot_mat=euler_to_matrix([cords[i-1,3],0,0])
            rot_mat=euler_to_matrix(cords[i-1,3],0,0)
            last_pose=label
            last_pose[:3]=rot_mat.dot(last_pose[:3])
            last_pose+=cords[i-1]
            last_pose[3] = (last_pose[3] + np.pi) % (2 * np.pi) - np.pi
            cords[i]=last_pose
    return cords.copy()

In [ ]:
_mean=train_label.mean(axis=0)
_std=train_label.std(axis=0)
_min=train_label.min(axis=0)
_max=train_label.max(axis=0)
_diff=(_max - _min)
if label_scale=='diff_div': _sub= 0;   _div=_diff
if label_scale=='diff_scl': _sub=_mean;_div=_diff
if label_scale=='norm_scl': _sub=_mean;_div=_std

def scl_label(_label):
    _label=(_label-_sub)/_div#[5,1,1.25,20,1,1]
    return _label
def unscl_label(_label):
    _label=(_label*_div)+_sub
    return _label
train_label=scl_label(train_label)
val_label=scl_label(val_label)
test_label=scl_label(test_label)
#_train_label=unscl_label(_train_label)

In [ ]:
fig,axes=plt.subplots(nrows=1,ncols=len(test_files),figsize=(15,3))
for i,[kk,vv] in enumerate(test_indx.items()):
    abs_label=get_label_abs_pose(unscl_label(test_label[vv[0]:vv[1]]))
    axes[i].plot(abs_label[:,0],abs_label[:,2])
    axes[i].set_title('test_'+str(kk))

fig,axes=plt.subplots(nrows=2,ncols=3,figsize=(15,5))
for i in range(no_of_pose):
    if time_step>2 :axes[i//3,i%3].plot(train_label[:,-1,i])
    else : axes[i//3,i%3].plot(train_label[: ,i])

### Generating train image path sequences

In [ ]:
def get_image_paths(img_root1,_files,_overlap=1):
    img_data_a=[]
    img_data_b=[]
    img_root2='\\image_2\\*'
    for start_point in start_points:
        for i in _files:
           img_q=deque(maxlen=time_step)
           for path in glob.glob(img_root1+str(i).zfill(2)+img_root2)[start_point:-1]:
                    img_q.append(path)
                    if len(img_q)==time_step:
                       img_data_a.append(list(img_q))
                       img_q=deque(list(img_q)[-_overlap:],maxlen=time_step)

    return np.array(img_data_a)
train_data=get_image_paths(img_root,train_files,_overlap=overlap)
val_data=get_image_paths(img_root,val_files,_overlap=overlap)
test_data=get_image_paths(img_root,test_files,_overlap=(time_step-1))

### Repeating data high rotation

In [ ]:
fig=plt.figure(figsize=(20,5))
plt.plot(train_label[:,3])
grads=np.gradient(train_label[:,3])
zc = np.diff(np.sign(grads))

def rolling_mean(_arr,_win):
    _d=_win//2
    n_arr=_arr.copy()
    __arr=_arr.copy()
    for i in range(len(__arr)):
        if i>_d:
            n_arr[i]=__arr[i-_d:i+_d].mean()
    return n_arr
_zc=rolling_mean(abs(zc),75)
plt.plot(_zc<0.3)
plt.plot(_zc)
plt.axhline(y=0.3,color='r')
plt.show()
fig=plt.figure(figsize=(20,5))
repeat_index=[False]+list(_zc<0.3)
plt.plot(train_label[repeat_index,3])
plt.show()

In [ ]:
if repeat_data==True:
    train_label=np.concatenate([train_label,train_label[repeat_index]],axis=0)
    train_data=np.concatenate([train_data,train_data[repeat_index]],axis=0)
train_label.shape,train_data.shape

In [ ]:
print(train_data[:2],'\n',test_data[:2])
print(len(train_data),len(train_label),len(test_data),len(test_label))

### Saving data and shuffling

In [ ]:
df=pd.DataFrame({'img_path':list(train_data),'label':list(train_label)})
df.to_csv(log_dir+exp_name+'data.csv')

In [ ]:
train_index=np.arange(len(train_data))
np.random.seed(seed)
np.random.shuffle(train_index)
val_index=np.arange(len(val_data))
#np.random.seed(seed)
#np.random.shuffle(val_index)

if load_random_vars==True:
    with open(random_vars_dir+'train_index.pkl','rb') as file:
         train_index=pickle.load(file)
            
if not os.path.exists(log_dir+exp_name+'vars/'): os.mkdir(log_dir+exp_name+'vars/')
with open(log_dir+exp_name+'vars/train_index.pkl','wb') as file:
     pickle.dump(train_index,file)

In [ ]:
_train_data=[]
_train_label=[]
for i in train_index:
    _train_data.append(train_data[i])
    _train_label.append(train_label[i])
_train_label=np.array(_train_label)
_val_data=[]
_val_label=[]
for i in val_index:
    _val_data.append(val_data[i])
    _val_label.append(val_label[i])
_val_label=np.array(_val_label)
len(_train_label),len(_val_label),len(test_label)

### Optical Flow Utility functions

In [ ]:

def hsv2bgr(hsv,scl=2):
    out=np.zeros((image_h//2,image_w//2,3),dtype=np.uint8)
    out[:,:,0]=hsv[:,:,0]*255
    out[:,:,1]=255
    out[:,:,2]=hsv[:,:,1]*255
    bgr=cv2.cvtColor(out,cv2.COLOR_HSV2BGR)
    return bgr

def bgr2hsv(bgr,scl=2):
    out=np.zeros((image_h//2,image_w//2,3),dtype=np.float32)
    out[:,:,0]=bgr[:,:,0]/255.0
    out[:,:,1]=1.0
    out[:,:,2]=bgr[:,:,1]/255.0
    bgr=cv2.cvtColor(out,cv2.COLOR_HSV2BGR)
    return bgr


def img2flow_vect(image,scl=2,_format='hsv'):
    image=cv2.resize(image,(image.shape[1]//scl,image.shape[0]//scl))
    if _format=='bgr':
        hsv=cv2.cvtColor(image,cv2.COLOR_BGR2HSV)
        hue=hsv[:,:,0]
        val=hsv[:,:,2]
    if _format=='hsv' :
        hue=hsv[:,:,0]
        val=hsv[:,:,2]
    ang=(hue/255)*2*np.pi
    vel=val/255
    fx = vel*np.cos(ang)
    fy = vel*np.sin(ang)
    flow=np.zeros((image.shape[0],image.shape[1],2),dtype=np.float64)
    flow[:,:,0]=fx#np.uint8(of_img[:,:,0]*180)
    flow[:,:,1]=fy#np.uint8(of_img[:,:,1]*255)
    
    return flow

def flow_vect2img(image,_format='hsv'):
    fx=image[:,:,0]
    fy=image[:,:,1]
    ang = np.arctan2(fy, fx)# + np.pi
    vel = np.sqrt(fx*fx+fy*fy)#/np.sqrt(2)
    ang=(ang/(2*np.pi))
    ang=np.clip(ang,0,0.5)

    if _format=='hsv':
        out=np.zeros((image.shape[0],image.shape[1],2),dtype=np.float32)
        out[:,:,0]=ang
        out[:,:,1]=vel
        #out[:,:,1]=cv2.normalize(vel,None,0,255,cv2.NORM_MIMMAX)
        
    if _format=='bgr':
        out=np.zeros((image.shape[0],image.shape[1],3),dtype=np.uint8)
        out[:,:,0]=ang*255
        out[:,:,1]=255
        out[:,:,2]=255*vel
        out=cv2.cvtColor(out,cv2.COLOR_HSV2BGR)
        
    return out

def get_flow_path(_path):
        _path=_path.replace('sequences','flow')
        _path=_path.replace('png','flow')
        _path=_path.replace('\\image_2','')
        return _path

def get_disp_path(_path):
        _path=_path.replace('sequences','disp')
        _path=_path.replace('png','disp')
        _path=_path.replace('\\image_2','')
        return _path
    
def get_seg_path(_path):
        _path=_path.replace('sequences','seg')
        _path=_path.replace('png','seg')
        _path=_path.replace('\\image_2','')
        return _path
    
def get_flow_vect(img1,img2):
    hsv = np.zeros_like(img1)
    hsv[...,1] = 255
    _img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    _img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(_img1,_img2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    return flow

def get_flow_hsv(img1,img2):
    hsv = np.zeros_like(img1)
    hsv[...,1] = 255
    _img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    _img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(_img1,_img2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    return hsv

### Data Batch Generator

In [ ]:
def resize_scl(image,scl):
    of_img=cv2.resize(image,(image_w//scl,image_h//scl))
    return of_img


def normalize_img(img):
    if _img_norm==True:
        #mean = (-0.1228108650837056, -0.12498114858829212, -0.1423079117403446)
        #std =  (0.3224294024021682, 0.3199054482966101, 0.31869854090215954)
        mean = (-0.19007764876619865, -0.15170388157131237, -0.10659445665650864)
        std =  (0.2610784009469139, 0.25729316928935814, 0.25163823815039915)
        img[:,:,0]=(img[:,:,0]-mean[0])/std[0]
        img[:,:,1]=(img[:,:,1]-mean[1])/std[1]
        img[:,:,2]=(img[:,:,2]-mean[2])/std[2]
    
    return img.copy()
    
def batch_generator(_data,_label):
    no_of_batch=len(_data)//batch_size
    batch_index=np.arange(no_of_batch)
    #np.random.seed(seed)
    #np.random.shuffle(batch_index)
    if time_step>2 :
        batch_data_a=np.zeros((batch_size,image_h,image_w,3))
        batch_data_b=np.zeros((batch_size,image_h,image_w,3))
        batch_flow=np.zeros((batch_size,time_step-1,image_h//2,image_w//2,_flow_chanel))
    else:
        batch_data_a=np.zeros((batch_size,image_h,image_w,3))
        batch_data_b=np.zeros((batch_size,image_h,image_w,3))
        batch_flow_1=np.zeros((batch_size,image_h//_flow_scale,image_w//_flow_scale,_flow_chanel))
        #batch_flow_2=np.zeros((batch_size,image_h//(_flow_scale*2),image_w//(_flow_scale*2),_chanel))
        #batch_flow_3=np.zeros((batch_size,image_h//(_flow_scale*4),image_w//(_flow_scale*4),_chanel))
        #batch_flow_4=np.zeros((batch_size,image_h//(_flow_scale*8),image_w//(_flow_scale*8),_chanel))
        #batch_flow_5=np.zeros((batch_size,image_h//(_flow_scale*16),image_w//(_flow_scale*16),_chanel))
        
        batch_disp_1=np.zeros((batch_size,image_h//_disp_scale,image_w//_disp_scale,1))
        
    while(True):
        for ind in batch_index:#range(no_of_batch):
            k=0
            for j in range(ind*batch_size,(ind+1)*batch_size):
                imgs=[]
                for p,fname in enumerate(_data[j]):
                    img_a=cv2.imread(fname)
                    img_a=cv2.resize(img_a,(image_w,image_h))
                    imgs.append(img_a)
                    if p>0:
                        flow_path=get_flow_path(str(_data[j][0]))
                        with open(flow_path,'rb') as file:
                            of_img=pickle.load(file)
                            if _flow_decompress==True:
                                of_img=zlib.decompress(of_img)
                                of_img=np.frombuffer(of_img,dtype=np.uint8).reshape(192,640,2)
                                of_img=(of_img/120-1)
                        
                        if _disp_or_seg=='disp': disp_path=get_disp_path(str(_data[j][0]))
                        if _disp_or_seg=='seg' : disp_path=get_seg_path(str(_data[j][0]))
                        with open(disp_path,'rb') as file:
                            disp_img=pickle.load(file)
                            
                                
                        #if math.isnan(of_img.mean())==True:
                        #    of_img=np.nan_to_num(of_img)
                        
                        if _flow_format=='hsv' :
                            of_img=flow_vect2img(of_img,_format='hsv')
                        #if flow_format='vect' :
                        #    pass
                        of_img=cv2.resize(of_img,(image_w//_flow_scale,image_h//_flow_scale))
                        disp_img=cv2.resize(disp_img,(image_w//_disp_scale,image_h//_disp_scale))
                        if time_step>2:
                            batch_data_a[k,p-1,:,:,:]=normalize_img((imgs[p-1]/255.0)-_img_offset)
                            batch_data_b[k,p-1,:,:,:]=normalize_img((imgs[p]/255.0)-_img_offset)
                            batch_flow_1[k,p-1,:,:,:]=of_img
                        else :
                            batch_data_a[k,:,:,:]=(imgs[p-1]/255.0)-_img_offset
                            batch_data_b[k,:,:,:]=(imgs[p]/255.0)-_img_offset
                            batch_flow_1[k,:,:,:]=of_img
                            #batch_flow_2[k,:,:,:]=resize_scl(of_img,_flow_scale*2)
                            #batch_flow_3[k,:,:,:]=resize_scl(of_img,_flow_scale*4)
                            #batch_flow_4[k,:,:,:]=resize_scl(of_img,_flow_scale*8)
                            #batch_flow_5[k,:,:,:]=resize_scl(of_img,_flow_scale*16)
                            batch_disp_1[k,:,:,0]=disp_img
                
                k+=1
            batch_trans_label=np.array(_label[ind*batch_size:(ind+1)*batch_size,:3],dtype=np.float32)
            batch_rot_label=np.array(_label[ind*batch_size:(ind+1)*batch_size,3:],dtype=np.float32)
            yield ([batch_data_a,batch_data_b],[batch_trans_label,batch_rot_label,batch_flow_1,batch_disp_1])#,batch_flow_2,batch_flow_3,batch_flow_4,batch_flow_5])

In [ ]:
train_batch = batch_generator(_train_data,_train_label)
val_batch   = batch_generator(_val_data,_val_label)
grad_batch   = batch_generator(_train_data[:batch_size*_grad_data_len],_train_label[:batch_size*_grad_data_len])
sample=next(train_batch)

for smp in sample[1]:
    print(smp.shape)


### Visualizing some random bathces

In [ ]:
if time_step>2 :seq=sample[0][0][0] #data,data1,batch_1
else : seq=sample[0]
fig,axes=plt.subplots(nrows=len(seq[0]),ncols=4,figsize=(15,15))
labels=sample[1][2:]
for i,[img_a,img_b,flow,disp] in enumerate(zip(seq[0],seq[1],labels[0],labels[1])):
    if _flow_format=='hsv' :
        bgr=hsv2bgr(flow)
    if _flow_format=='vect' :
        bgr=flow_vect2img(flow,_format='bgr')
    axes[i,0].imshow(img_a+0.5)
    axes[i,1].imshow(img_b+0.5)
    axes[i,2].imshow(bgr[:,:,:])
    axes[i,3].imshow(disp[:,:,0])
    #axes[i,3].imshow(depth,cmap='gray')

### Loss Functions and model defining

In [ ]:
def custom_loss_flow(y_true, y_pred):
      y_true=tf.cast(y_true,tf.float32)
      y_pred=tf.cast(y_pred,tf.float32)
    
      loss=tf.square(tf.subtract(y_true,y_pred))
      abs_loss=tf.reduce_mean(tf.abs(tf.subtract(y_true,y_pred)),axis=[1,2,3])
      if _flow_loss=='rmse' : loss=tf.sqrt(tf.reduce_mean(loss,axis=[1,2,3]))
      if _flow_loss=='mse' : loss=tf.reduce_mean(loss,axis=[1,2,3])
      return loss,abs_loss 

def custom_loss_flow_log(y_true, y_pred):
      y_true=tf.cast(y_true,tf.float32)
      if _flow_format=='hsv':
                      y_true=tf.clip_by_value(tf.cast(y_true,tf.float32),0.0,1.0)
                      y_pred=tf.clip_by_value(tf.cast(y_pred,tf.float32),0.0,1.0)
                      _add_offset=1
      if _flow_format=='vect':
                      y_true=tf.clip_by_value(tf.cast(y_true,tf.float32),-1.0,1.0)
                      y_pred=tf.clip_by_value(tf.cast(y_pred,tf.float32),-1.0,1.0)
                      _add_offset=2
      y_true=tf.math.log(y_true+_add_offset)
      y_pred=tf.math.log(y_pred+_add_offset)
        
      loss=tf.square(tf.subtract(y_true,y_pred))
      abs_loss=tf.reduce_mean(tf.abs(tf.subtract(y_true,y_pred)),axis=[1,2,3])
      if _flow_loss=='msle':  loss=tf.reduce_mean(loss,axis=[1,2,3])
      if _flow_loss=='rmsle': loss=tf.sqrt(tf.reduce_mean(loss,axis=[1,2,3]))
      return loss,abs_loss 

def custom_loss_disp(y_true, y_pred):
      y_true=tf.cast(y_true,tf.float32)
      y_pred=tf.cast(y_pred,tf.float32)
    
      loss=tf.square(tf.subtract(y_true,y_pred))
      abs_loss=tf.reduce_mean(tf.abs(tf.subtract(y_true,y_pred)),axis=[1,2,3])
      if _disp_loss=='rmse' : loss=tf.sqrt(tf.reduce_mean(loss,axis=[1,2,3]))
      if _disp_loss=='mse' : loss=tf.reduce_mean(loss,axis=[1,2,3])
      return loss
    
def custom_loss_disp_log(y_true, y_pred):
      y_true=tf.cast(y_true,tf.float32)
      #y_pred=tf.clip_by_value(tf.cast(y_pred,tf.float32),0,10)
      y_true=tf.math.log(y_true+1)
      y_pred=tf.math.log(y_pred+1)
      loss=tf.square(tf.subtract(y_true,y_pred))
      abs_loss=tf.reduce_mean(tf.abs(tf.subtract(y_true,y_pred)),axis=[1,2,3])
      if _disp_loss=='msle':  loss=tf.reduce_mean(loss,axis=[1,2,3])
      if _disp_loss=='rmsle': loss=tf.sqrt(tf.reduce_mean(loss,axis=[1,2,3]))
      return loss,abs_loss 

def custom_loss_trans(y_true,y_pred):
    abs_loss_trans=tf.reduce_mean(tf.abs(tf.subtract(y_true,y_pred)),axis=-1)
    if _pose_loss=='mae':
        loss_trans=tf.reduce_mean(tf.abs(tf.subtract(y_true,y_pred)),axis=-1)
    if _pose_loss=='mse':
        loss_trans=tf.reduce_mean(tf.square(tf.subtract(y_true,y_pred)),axis=-1)
    if _pose_loss=='rmse':
        loss_trans=tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_true,y_pred)),axis=-1))
    return loss_trans,abs_loss_trans

def custom_loss_rot(y_true,y_pred):
    abs_loss_rot=tf.reduce_mean(tf.abs(tf.subtract(y_true,y_pred)),axis=-1)
    if _pose_loss=='mae':
        loss_rot=tf.reduce_mean(tf.abs(tf.subtract(y_true,y_pred)),axis=-1)
    if _pose_loss=='mse':
        loss_rot=tf.reduce_mean(tf.square(tf.subtract(y_true,y_pred)),axis=-1)
    if _pose_loss=='rmse':
        loss_rot=tf.sqrt(tf.reduce_mean(tf.square(tf.subtract(y_true,y_pred)),axis=-1))
    return loss_rot,abs_loss_rot

N=64//scale
def _TDS(_layer):
    return _layer

if time_dis==True:
    TDS=TimeDistributed
else :
    TDS=_TDS
    
    
    
def ConvNet(x,classes=3,i=''):
    _convs=[]
    _par=_net_par
    for i in range(_par._num_shared_layer):
        _name='block_'+str(_par._block[i])+'_conv_'+str(_par._nconv[i])
        x = TDS(Conv2D(_par._filters[i], (_par._kernels[i], _par._kernels[i]), padding='same',strides=_par._strides[i],kernel_regularizer=_par._kernel_regularizer,kernel_initializer=_par._kernel_initializer, name=_name))(x)
        if _par._batch_norm[i]: x = BatchNormalization()(x)
        if _par._act[i]==1:  
                          if _par._leaky_relu==True: x = LeakyReLU(alpha=0.1)(x)
                          if _par._relu==True: x = ReLU()(x)
        if _par._drop_out[i]>0.0 : x = Dropout(_par._drop_out[i],seed=_par.drop_seed)(x)
        if _par._nconv[i]==2 or _par._block[i]==2: _convs.append(x)
    _conv_flow=x
    _conv_disp=x
    x = TDS(Flatten())(x)
    #return x

    
    
    _convs=list(reversed(_convs[:4]))#conv6_1,conv5_1,conv4_1,conv3_1,conv2_1
    flows=[]
    disps=[]
    
    for i in range(4):

        conv_trans=Conv2DTranspose(_par._deconv_filters[i], (4,4), strides=(2, 2), padding='same', name='conv_trans_flow_'+str(i))(_conv_flow)
        if _par._deconv_act[i] :
            if _par._leaky_relu==True: conv_trans= LeakyReLU(alpha=0.1)(conv_trans)
            if _par._relu==True: conv_trans= ReLU()(conv_trans)
        flow = Conv2D(_par._flow_chanel, (3, 3),  padding='same',strides=1, name='conv_flow'+str(i))(_conv_flow)
        flow = Conv2DTranspose(_par._flow_chanel, (4,4), strides=(2, 2), padding='same', name='flow_'+str(i))(flow)
        flows.append(flow)
        #if i==-1: _conv_flow = concatenate([_convs[i],conv_trans], axis = 3)
        _conv_flow = concatenate([_convs[i],conv_trans,flows[i]], axis = 3)
            
        
    i+=1
    flow = Conv2D(_par._flow_chanel, (3, 3),  padding='same',strides=1, name='conv_flow'+str(i))(_conv_flow)
    flow = Conv2DTranspose(_par._flow_chanel, (4,4), strides=(2, 2), padding='same', name='flow_'+str(i))(flow)
    flows.append(flow)
    flows=list(reversed(flows))
    
    #return flows,x
    
    for i in range(4):

        conv_trans=Conv2DTranspose(_par._deconv_filters[i], (4,4), strides=(2, 2), padding='same', name='conv_trans_disp'+str(i))(_conv_disp)
        if _par._deconv_act[i] :
            if _par._leaky_relu==True: conv_trans= LeakyReLU(alpha=0.1)(conv_trans)
            if _par._relu==True: conv_trans= ReLU()(conv_trans)
        disp = Conv2D(_par._disp_chanel, (3, 3),  padding='same',strides=1, name='conv_disp'+str(i))(_conv_disp)
        disp = Conv2DTranspose(_par._disp_chanel, (4,4), strides=(2, 2), padding='same', name='disp_'+str(i))(disp)
        disps.append(disp)
        #if i==-1: _conv = concatenate([_convs[i],conv_trans], axis = 3)
        _conv_disp = concatenate([_convs[i],conv_trans,disps[i]], axis = 3)
            
        
    i+=1
    disp = Conv2D(_par._disp_chanel, (3, 3),  padding='same',strides=1, name='conv_disp'+str(i))(_conv_disp)
    disp = Conv2DTranspose(_par._disp_chanel, (4,4), strides=(2, 2), padding='same', name='disp_'+str(i))(disp)
    disp=ReLU()(disp)
    disps.append(disp)
    disps=list(reversed(disps))
    
    #return flows,x
    return x,flows[0],disps[0]

if _flow_loss=='rmsle' or _flow_loss=='msle': _flow_loss_func=custom_loss_flow_log
if _flow_loss=='rmse' or _flow_loss=='mse': _flow_loss_func=custom_loss_flow
if _disp_loss=='rmsle' or _disp_loss=='msle': _disp_loss_func=custom_loss_disp_log
if _disp_loss=='rmse' or _disp_loss=='mse': _disp_loss_func=custom_loss_disp
    
loss_funcs_dict={'trans':custom_loss_trans,
                 'rot':custom_loss_rot,
                 'flow_4':_flow_loss_func,
                 'disp_4':_disp_loss_func}
                 #'flow_3':custom_loss_flow,
                 #'flow_2':custom_loss_flow,
                 #'flow_1':custom_loss_flow,
                 #'flow_0':custom_loss_flow} 


def PRF_NET(input_shape=(224,224,2)):
    _input_a=Input(shape=input_shape)
    _input_b=Input(shape=input_shape) 
    #_input_c=Input(shape=[input_shape[0]//_depth_scale,input_shape[1]//_depth_scale,3])
    #_depth=Input(shape=[input_shape[0]//_depth_scale,input_shape[1]//_depth_scale])
    _input=concatenate([_input_a,_input_b],axis=3)
    x,flow,disp=ConvNet(_input,i=0)
    _par=_net_par
    if time_dis==True:
               x = LSTM(_par._lstm_size,kernel_initializer=_par._kernel_initializer,recurrent_initializer=_par._kernel_initializer,return_sequences=True )(x)
               if _par._rot_drop_out[0]>0.0 : x = Dropout(_par._drop_out[10],seed=_par.drop_seed)(x)
               x = LSTM(_par._lstm_size,kernel_initializer=_par.kernel_initializer,recurrent_initializer=_par._kernel_initializer,return_sequences=True)(x)
               if _par._rot_drop_out[1]>0.0 : x = Dropout(_par._drop_out[11],seed=_par.drop_seed)(x)
    else :
               trans=x
               for i in range(_par._num_trans_layers):
                   trans = Dense(_par._trans_layer_nodes[i],kernel_regularizer=_par._kernel_regularizer, name='dense_1.'+str(i+1))(trans)
                   if _par._trans_act[i]==1 :
                              if _par._relu==True: trans = ReLU()(trans)
                              if _par._leaky_relu==True: trans = LeakyReLU(alpha=0.1)(trans)
                   if _par._trans_drop_out[i]>0.0 : trans = Dropout(_par._drop_out[i],seed=_par.drop_seed)(trans)
                
               trans = Dense(no_of_pose//2,kernel_regularizer=_par._kernel_regularizer, name='trans')(trans)
            
               rot=x
               for i in range(_par._num_rot_layers):
                   rot = Dense(_par._rot_layer_nodes[i],kernel_regularizer=_par._kernel_regularizer, name='dense_2.'+str(i+1))(rot)
                   if _par._rot_act[i]==1:
                                     if _par._relu==True: rot = ReLU()(rot)
                                     if _par._leaky_relu==True: rot = LeakyReLU(alpha=0.1)(rot)
                   if _par._rot_drop_out[i]>0.0 : rot = Dropout(_par._drop_out[i],seed=_par.drop_seed)(rot)
                
               rot = Dense(no_of_pose//2,kernel_regularizer=_par._kernel_regularizer, name='rot')(rot)
    
    model = Model([_input_a,_input_b], [trans,rot,flow,disp], name='network')
    model.summary()
    return model

    
    
if time_dis==True : model = PRF_NET(input_shape=[time_step-1,image_h, image_w,3])
else : model = PRF_NET(input_shape=[image_h, image_w,3])
for layer in model.layers:
    if 'disp' in layer.name: layer.trainable=_train_disp
    if 'flow' in layer.name: layer.trainable=_train_flow
    print(layer.name,layer.trainable)
model.summary()
plot_model(model,to_file=log_dir+exp_name+'model.png')

if load_init_wts==True:
       model.load_weights(random_vars_dir+'init_weights.hdf5')
model.save(log_dir+exp_name+'vars/init_weights.hdf5')

        
if pre_trained_wts_path !=None: 
        model.load_weights(pre_trained_wts_path)
if pre_trained_opt_path !=None: 
        #model._make_train_function()
        with open(pre_trained_opt_path, 'rb') as f:
            curr_optimizer = pickle.load(f)    

In [ ]:
layer_count=0
if len(wt_path) != 0:
    weight_file = h5py.File(wt_path, 'r')
    curr_layer_names=[lyr.name for i,lyr in enumerate(model.layers)]
    saved_layer_names=list(weight_file['model_weights'].keys())
    
    for saved_name in saved_layer_names:
        try :shp = weight_file['model_weights'][saved_name][saved_name]['kernel:0'].shape
        except: continue
        if saved_name in curr_layer_names:
            weights=weight_file['model_weights'][saved_name][saved_name]['kernel:0']
            biases=weight_file['model_weights'][saved_name][saved_name]['bias:0']
            layer=model.get_layer(saved_name)
            layer.set_weights([weights,biases])
            layer_count+=1
            if saved_name[:5] != 'block': layer.trainable = False
            print(saved_name,shp,layer.trainable)

In [ ]:
if train==True:
    #class loss_history(tf.keras.callbacks.Callback):
    class loss_logger():
        def on_train_begin(self, logs={}):
            if pre_trained_log_path ==None: 
                self.fe=open(log_dir+exp_name+'loss_epoch.csv','w+')
            else : 
                old_log=open(pre_trained_log_path,'r')
                old_log_data = old_log.read()
                old_log.close()
                self.fe=open(log_dir+exp_name+'loss_epoch.csv','w+')
                for line in old_log_data:
                        self.fe.write(line)
            self.fe.close()
            self.columns=[]
            
            
        def on_epoch_begin(self, epoch, logs={}):
            self.epoch=epoch            
            
        def on_epoch_end(self, epoch, logs={},grads={},calculated_grads=[],_curr_opt=[]):
            if epoch==1:
                loss_str=''
                for _item in logs.keys():
                    loss_str+=str(_item)+','
                    self.columns.append(_item)
                loss_str=loss_str[:-1]+'\n'
            else:
                loss_str=''
                
            for _item in self.columns:
                loss_str+=str(logs[_item])+','
            loss_str=loss_str[:-1]+'\n'
            
            with open(log_dir+exp_name+'loss_epoch.csv','a+') as fe:
                fe.write(loss_str)
            if not os.path.exists(log_dir+exp_name+'calc_grads'): os.mkdir(log_dir+exp_name+'calc_grads')
            if not os.path.exists(log_dir+exp_name+'grads'): os.mkdir(log_dir+exp_name+'grads')
            
            if epoch%_grad_log_epoch==0 or epoch==1:
                
                _t_wts={};_r_wts={};_f_wts={};
                
                _t_wts=grads['trans']
                _r_wts=grads['rot']
                _f_wts=grads['flow_4']
                _d_wts=grads['disp_4']
                
                with open(log_dir+exp_name+'grads\\'+'trans_'+str(epoch)+'.grads','wb') as file:
                        pickle.dump(_t_wts,file)
                with open(log_dir+exp_name+'grads\\'+'rot_'+str(epoch)+'.grads','wb') as file:
                        pickle.dump(_r_wts,file)
                if _train_flow==True:
                    with open(log_dir+exp_name+'grads\\'+'flow_'+str(epoch)+'.grads','wb') as file:
                            pickle.dump(_f_wts,file)
                if _train_disp==True:
                    with open(log_dir+exp_name+'grads\\'+'disp_'+str(epoch)+'.grads','wb') as file:
                            pickle.dump(_d_wts,file)
                        
            if epoch%_calc_grad_log_epoch==0 or epoch==1:
                with open(log_dir+exp_name+'calc_grads\\'+'calc_grads_'+str(epoch)+'.grads','wb') as file:
                        for key in calculated_grads.keys():
                            if type(calculated_grads[key]) == list:
                                for i in range(len(calculated_grads[key])):
                                    if calculated_grads[key][i] != None:
                                         calculated_grads[key][i]= calculated_grads[key][i].numpy() 
                            else:
                                calculated_grads[key]= calculated_grads[key].numpy()  
                        pickle.dump(calculated_grads,file)
                
            
            #symbolic_weights = getattr(_opt, 'weights')
            #weight_values = K.batch_get_value(symbolic_weights)
            if not os.path.exists(log_dir+exp_name+'weights'): os.mkdir(log_dir+exp_name+'weights')
            if (epoch)%_all_wts_log_epoch==0:
                model.save(log_dir+exp_name+'weights\\'+str(epoch)+'.hdf5')
                if _save_opts==True:
                    if not os.path.exists(log_dir+exp_name+'opts'): os.mkdir(log_dir+exp_name+'opts')
                    with open(log_dir+exp_name+'opts\\'+str(epoch)+'.pkl', 'wb') as f:
                            pickle.dump(_curr_opt, f)
 
        
    
    
    

### Multi task learning with Auxiliary task

In [ ]:
@tf.function
def train_step_main(_images, _labels,_task):
#def train_step_main(_images, _labels,_loss_dict,_std_dict,_grad_dict,_task,total_gradients):
    _mtp=train_params_tracker(_task)
    _loss_funcs=list(loss_funcs_dict.keys())
    _n=_loss_funcs.index(_task)
    with tf.GradientTape() as tape:
        _predictions = model(_images, training=True)
        lbl,pred,loss_name=_labels[_n],_predictions[_n],_loss_funcs[_n]

        loss_func=loss_funcs_dict[loss_name]
        loss_weight=loss_weights_dict[loss_name]

        _batch_loss,_abs_loss=loss_func(lbl, pred)
        _std_loss=tf.math.reduce_std(_abs_loss)
        _abs_loss=tf.math.reduce_mean(_abs_loss)
        
        _loss = tf.reduce_mean(_batch_loss)
        _gradients = tape.gradient(_loss, model.trainable_variables) 
        
        _mtp._loss_dict[loss_name+'_loss']=_loss
        _mtp._loss_dict['train_loss']=loss_weight*_loss
        _mtp._std_dict[loss_name+'_std'] = _std_loss 
        _mtp._grad_dict[loss_name+'_grad'] = _gradients
        _mtp._grad_dict[loss_name+'_abs_loss'] = _abs_loss
    del tape
    for i in range(len(_mtp.total_gradients)):
        if _gradients[i]!=None: _mtp.total_gradients[i]= tf.multiply(loss_weight,_gradients[i])

    return [_mtp.__dict__]

class slope_handler():
    
    def __init__(self):
        self.rel_loss_logger=np.zeros((epochs+1,len(_init_loss_wts)),dtype=np.float32)
        self.rel_reward_logger=np.zeros((epochs+1,len(_init_loss_wts)),dtype=np.float32)
        self.rel_loss_1st_grad_logger=np.ones((epochs+1,len(_init_loss_wts)),dtype=np.float32)     
        self.rel_loss_2nd_grad_logger=np.ones((epochs+1,len(_init_loss_wts)),dtype=np.float32)
        self.prev_rel_loss=np.ones(len(_init_loss_wts),dtype=np.float32)
        self.prev_rel_reward=np.zeros(len(_init_loss_wts),dtype=np.float32)
        
        self._rel_loss={}
        self._rel_reward={}
        self._1st_grad={}
        self._2nd_grad={}
        self.update(init=True)
        
    def get_nth_grad(self,nth,_epoch):
        _grad=self.rel_reward_logger[:_epoch+1]
        for i in range(nth):
            _grad=np.gradient(_grad,axis=0)
            #if (i+1)%2 !=0:
            #    _grad=-1*_grad
        _grad=_grad+_mtl_loss_slope_center
        return _grad
    
    def update(self,_epoch=1,_curr_rel_loss=[],init=False):
        if init==True:
            for loss_name in loss_funcs_dict.keys():
                self._rel_loss[loss_name]=_mtl_loss_center+0.5
                self._rel_reward[loss_name]=0.0
                self._1st_grad[loss_name]=1.0
                self._2nd_grad[loss_name]=1.0
            
        else:
            curr_rel_reward=1-np.array(list(_curr_rel_loss.values()))
            self.rel_reward_logger[_epoch,:]=_mtl_loss_slope_alpha*curr_rel_reward+(1-_mtl_loss_slope_alpha)*self.prev_rel_reward
            self.prev_rel_reward=self.rel_reward_logger[_epoch,:]
            
            curr_rel_loss=np.array(list(_curr_rel_loss.values()))
            self.rel_loss_logger[_epoch,:]=_mtl_loss_slope_alpha*curr_rel_loss+(1-_mtl_loss_slope_alpha)*self.prev_rel_loss
            self.prev_rel_loss=self.rel_loss_logger[_epoch,:]
            
            #Getting Each values for current epoch
            self.rel_loss_logger[_epoch,:]=self.rel_loss_logger[_epoch,:]+(_mtl_loss_center-0.5)
            #self.rel_reward_logger[_epoch,:]=self.rel_reward_logger[_epoch,:]
            self.rel_loss_1st_grad_logger=self.get_nth_grad(1,_epoch)
            self.rel_loss_2nd_grad_logger=self.get_nth_grad(2,_epoch)
            
            #Saving each values for current epoch for using in current training step
            for loss_ind,loss_name in enumerate(list(loss_funcs_dict.keys())):
                self._rel_loss[loss_name]=self.rel_loss_logger[_epoch,loss_ind]
                self._rel_reward[loss_name]=self.rel_reward_logger[_epoch,loss_ind]
                self._1st_grad[loss_name]=self.rel_loss_1st_grad_logger[_epoch,loss_ind]
                self._2nd_grad[loss_name]=self.rel_loss_2nd_grad_logger[_epoch,loss_ind]
                   
            
        
        
    def log(self,_epoch=1):
        loss_slope_dict=loss_dict_obj()
        for loss_ind,loss_name in enumerate(loss_funcs_dict.keys()):
                loss_slope_dict[loss_name+'_0th_reward']=self.rel_reward_logger[_epoch,loss_ind]
                loss_slope_dict[loss_name+'_0th_loss']=self.rel_loss_logger[_epoch,loss_ind]
                loss_slope_dict[loss_name+'_1st_grad']=self.rel_loss_1st_grad_logger[_epoch,loss_ind]
                loss_slope_dict[loss_name+'_2nd_grad']=self.rel_loss_2nd_grad_logger[_epoch,loss_ind]
        return loss_slope_dict
    
def cosine_similarity_tf(x,y):
    dot_prod=tf.reduce_sum(x*y)
    mag_x=tf.sqrt(tf.reduce_sum(tf.square(x)))
    mag_y=tf.sqrt(tf.reduce_sum(tf.square(y)))
    _res=tf.truediv(dot_prod,tf.multiply(mag_x,mag_y))
    #_res=tf.math.acos(_res)
    return _res

class cosine_handler():
        def __init__(self):
            self.cosines={}
            self.update()
                
        def update(self,_curr_cosine=None):
            for loss_name in (loss_funcs_dict.keys()):
                if _curr_cosine==None:
                    self.cosines[loss_name]=tf.ones((len(model.trainable_variables)),dtype=tf.float32)
                else:
                    try: self.cosines[loss_name]=_curr_cosine[loss_name]
                    except KeyError: pass
                    
        def get_name(self,layer_name,loss_name):
            _name=layer_name[:-2].replace('/','_')
            _name=_name.replace('block','layer')
            _name=_name.replace('_conv_','')
            _name=loss_name+'_'+_name
            return _name
        
        def log(self,_curr_cosines):
            cosine_log_dict=loss_dict_obj()
            for loss_ind,loss_name in enumerate(list(loss_funcs_dict.keys())[2:]):
                    for layer_name in grad_log_layers:
                        if layer_name[:5]=='block':
                            layer_ind=layer_name_dict[layer_name]
                            _name=self.get_name(layer_name,loss_name)
                            cosine_log_dict[_name]=self.cosines[loss_name][layer_ind].numpy()
                            
            return cosine_log_dict
        
@tf.function        
def get_grad_var(_images, _labels,_task):
        _loss_funcs=list(loss_funcs_dict.keys())
        _n=_loss_funcs.index(_task)
        
        _grad_variance=[None]*36
        _all_grads=[None]*batch_size
        
        with tf.GradientTape(persistent=True) as tape: 
            _predictions = model(_images, training=True)

            lbl,pred,loss_name=_labels[_n],_predictions[_n],_loss_funcs[_n]
            loss_func=loss_funcs_dict[loss_name]

            _batch_loss,_abs_loss_b=loss_func(lbl, pred)
            _loss_b = tf.reduce_mean(_batch_loss)

            _grads_m=tape.gradient(_loss_b, model.trainable_variables[:36])
            
            for k in range(batch_size):
                _grads=tape.gradient(_batch_loss[k], model.trainable_variables[:36]) 
                _all_grads[k]=_grads
                
        del tape                      
        for _grads in _all_grads:
            for k,(_g1,_g2) in enumerate(zip(_grads_m,_grads)):
                if _grad_variance[k] != None : _grad_variance[k]=_grad_variance[k]+(1/batch_size)*tf.square(_g1-_g2)
                else : _grad_variance[k]= (1/batch_size)*tf.square(_g1-_g2)
                    
        return _grad_variance
            
@tf.function
def train_step_aux(_images, _labels,_task,_mtp,_grad_var_dict):
        _atp=train_params_tracker(_task)
        _loss_funcs=list(loss_funcs_dict.keys())
        _main_task=_loss_funcs[_mtl_main_task_index]
        _n=_loss_funcs.index(_task)
        _std_loss_a=_mtp['_std_dict'][_main_task+'_std']
        _abs_loss_a=_mtp['_grad_dict'][_main_task+'_abs_loss']
        _gradients=_mtp['_grad_dict'][_main_task+'_grad']
        _grad_var_a=_grad_var_dict[_main_task]
        _grad_var_b=_grad_var_dict[_task]
        
        with tf.GradientTape() as tape: 
            _predictions = model(_images, training=True)

            lbl,pred,loss_name=_labels[_n],_predictions[_n],_loss_funcs[_n]
            loss_func=loss_funcs_dict[loss_name]
            loss_weight=loss_weights_dict[loss_name]

            _batch_loss,_abs_loss_b=loss_func(lbl, pred)
            _std_loss_b=tf.math.reduce_std(_abs_loss_b)
            _abs_loss_b=tf.math.reduce_mean(_abs_loss_b)

            _loss_b = tf.reduce_mean(_batch_loss)
            _grads=tape.gradient(_loss_b, model.trainable_variables) 

            _atp._loss_dict[loss_name+'_loss']=_loss_b
            _atp._loss_dict['train_loss']=loss_weight*_loss_b
            _atp._std_dict[loss_name+'_std'] = _std_loss_b
            _atp._grad_dict[loss_name+'_grad'] = _grads

        del tape
        if _mtl_aux_grad_algo=='v2' or _mtl_aux_grad_algo=='v3'or _mtl_use_cosine_v4==True:
                _cos=[]
                for k in range(len(_atp.total_gradients)):
                    if _gradients[k] != None and _grads[k] != None:
                        g1=tf.reshape(_gradients[k],[-1])
                        g2=tf.reshape(_grads[k],[-1])
                        _cos.append(cosine_similarity_tf(g1,g2))
                    else:
                        _cos.append(0.0)

                curr_cosine=tf.cast(_cos,dtype=tf.float32)#+1 check
                curr_cosine=(curr_cosine-tf.reduce_mean(curr_cosine))/tf.abs(tf.reduce_max(curr_cosine)-tf.reduce_min(curr_cosine))
                curr_cosine=_mtl_cosine_use_fact*curr_cosine+_mtl_cos_center
                #cosines.update(loss_name,curr_cosine)
                if _mtl_cosine_avg=='eqn_1' : _cosines=(1-_mtl_cos_alpha)*cosines.cosines[loss_name]+_mtl_cos_alpha*curr_cosine
                if _mtl_cosine_avg=='eqn_2' : _cosines=_mtl_cos_alpha*cosines.cosines[loss_name]+curr_cosine[loss_name]
                _atp.updated_cosines[loss_name]=_cosines
        _beta_layer_log=[];_beta_node_log=[]
        _beta_conv_log=[];_coef_log=[];_var_a_log=[];_var_b_log=[];_grads_b_log=[]
        
        for k in range(len(_atp.total_gradients)):
                if _gradients[k] != None and _grads[k] != None:

                        if _mtl_aux_grad_algo=='v1':
                                _beta=loss_weight

                        
                        if _mtl_aux_grad_algo=='v4':
                            
                                _min_std=10e-20

                                if _mtl_get_variance_from=='calc':   
                                    _var_a=_grad_var_a[k]+_min_std
                                    _var_b=_grad_var_b[k]+_min_std

                                if _mtl_use_factors=='default':
                                    _ratio_fact=_var_b/_var_a
                                    if _mtl_use_std_ratio==True:
                                        _ratio_fact=tf.sqrt(_ratio_fact)
                                    _coef=tf.square((_gradients[k]-_grads[k]))
                                    _coef=_coef/_var_a
                                    _coef=tf.exp(-0.5*_coef)
                                    _beta_conv=tf.multiply(_ratio_fact,_coef) 
                                    

                                elif _mtl_use_factors=='prob_only':
                                    _coef=tf.square((_gradients[k]-_grads[k]))
                                    _coef=_coef/_var_a
                                    _coef=tf.exp(-0.5*_coef)
                                    _std_fact=1/(tf.sqrt(2*np.pi)*tf.sqrt(_var_a))
                                    _beta_conv=tf.multiply(_std_fact,_coef) 
                                    if _mtl_take_log_prob==True: 
                                        _beta_conv=tf.math.log(_beta_conv+10e-20)

                                else:
                                    print('Give a valid value for _mtl_use_factors instead of ',_mtl_use_factors)
                                    return None

                                if _mtl_use_mode_v4=='mode-1' and _mtl_use_factors != 'combine_gaussian':
                                    _beta_conv=loss_weight*_beta_conv
                                if _mtl_use_cosine_v4==True:
                                    _beta_conv=tf.multiply(_beta_layer,_beta_conv)
                                
                                

                                _beta_conv_log.append(_beta_conv)
                                _var_a_log.append(_var_a)
                                _var_b_log.append(_var_b)
                                _coef_log.append(_coef)
                                
                                if _mtl_beta_norm_method=='min_max': 
                                    _beta_conv=(_beta_conv-tf.reduce_mean(_beta_conv))/(tf.reduce_max(_beta_conv)-tf.reduce_min(_beta_conv))
                                    _beta_conv=_mtl_beta_use_fact*_beta_conv+1.0
                                if _mtl_beta_norm_method=='std':     
                                    _beta_conv=(_beta_conv-tf.reduce_mean(_beta_conv))/tf.math.reduce_std(_beta_conv)
                                    _beta_conv=_mtl_beta_use_fact*_beta_conv+1.0
                                    
                        if _mtl_aux_grad_algo=='v1':_atp.total_gradients[k]=tf.multiply(_beta,_grads[k])# check ***
                        elif _mtl_aux_grad_algo=='v2':_atp.total_gradients[k]=tf.multiply(_beta_layer,_grads[k])# check ***
                        elif _mtl_aux_grad_algo=='v3':_atp.total_gradients[k]=tf.multiply(_beta_node,_grads[k])# check ***
                        elif _mtl_aux_grad_algo=='v4':_atp.total_gradients[k]=tf.multiply(_beta_conv,_grads[k])# check ***
                        elif _mtl_aux_grad_algo=='v4' and _mtl_use_factors=='combine_gaussian':
                                 _atp.total_gradients[k] = _gradients[k]+tf.multiply(_beta_conv,(_grads[k]-_gradients[k]))
                                 _atp.total_gradients[k] = loss_weight*_atp.total_gradients[k]
                        else: print('Give a valid value for _mtl_aux_grad_algo :',_mtl_aux_grad_algo,'or _mtl_use_factors :',_mtl_use_factors)
                
                elif _grads[k] != None:
                     _atp.total_gradients[k]= loss_weight*_grads[k]
                 
        
        
        if _mtl_aux_grad_algo=='v2':
            _atp._grad_dict['beta_layer_mask']=_beta_layer_log
        if _mtl_aux_grad_algo=='v3':
            _atp._grad_dict['beta_node_mask']=_beta_node_log
        if _mtl_aux_grad_algo=='v4':
            _atp._grad_dict[loss_name+'_beta_conv_mask']=_beta_conv_log
            _atp._grad_dict[loss_name+'_var_a']=_var_a_log
            _atp._grad_dict[loss_name+'_var_b']=_var_b_log
            if _mtl_use_factors=='importance_sample':
                    _atp._grad_dict[loss_name+'_grad_b']=_grads_b_log
            #_atp._grad_dict[loss_name+'_coef']=_coef_log
        return [_atp.__dict__]


def apply_clipped_grads(total_gradients):
      if _grad_clip != 0:
        for  _layer in grad_clip_layers.keys():
                 _clip_value=grad_clip_layers[_layer]
                 ind=layer_name_dict[_layer]
                 total_gradients[ind]= tf.clip_by_value(total_gradients[ind],-_clip_value,_clip_value) 

      curr_optimizer.apply_gradients(zip(total_gradients, model.trainable_variables))
      return 1

class train_params_tracker():
    def __init__(self,_task=None):
        self.total_gradients=[None]*len(model.trainable_variables)
        self.updated_cosines={}
        if _task != None :self.updated_cosines[_task]=tf.ones((len(model.trainable_variables)),dtype=tf.float32)
        self._grad_dict={}
        self._std_dict={}
        self._loss_dict={'train_loss':0.0,'rot_loss':0.0,'trans_loss':0.0,'flow_4_loss':0.0,'disp_4_loss':0.0}
        
    def update(self,obj):
        for i,(gr1,gr2) in enumerate(zip(self.total_gradients,obj['total_gradients'])):
            if gr1 ==None: gr1=0.0
            if gr2 ==None: gr2=0.0
            self.total_gradients[i]=gr1+gr2
        for key in self._loss_dict.keys():
            self._loss_dict[key]+=obj['_loss_dict'][key]
        self.updated_cosines.update(obj['updated_cosines'])
        self._grad_dict.update(obj['_grad_dict'])
        self._std_dict.update(obj['_std_dict'])
    def update_all(self,list_obj):
        for obj in list_obj:
            self.update(obj)
        
#@tf.function
def train_step_mtl(_images,_labels,_grad_var_dict={}):
    _updated_train_params=train_params_tracker()
    _train_params=[]      
    _train_params+=train_step_main(_images,_labels,'trans')
    _train_params+=train_step_main(_images,_labels,'rot')
    if _train_flow==True: _train_params+=train_step_aux(_images, _labels,'flow_4',_train_params[_mtl_main_task_index],_grad_var_dict)
    if _train_disp==True: _train_params+=train_step_aux(_images, _labels,'disp_4',_train_params[_mtl_main_task_index],_grad_var_dict)
    _updated_train_params.update_all(_train_params)
    del _train_params
    apply_clipped_grads(_updated_train_params.total_gradients)
    _grad_dict = _updated_train_params._grad_dict
    updated_cosines= _updated_train_params.updated_cosines
    _loss_dict = _updated_train_params._loss_dict
    tg=_updated_train_params.total_gradients
    del _updated_train_params
    return _loss_dict,_grad_dict,updated_cosines#_train_params[0]

### Traininig Block

In [ ]:
@tf.function
def train_step(images, labels,loss_funcs_dict,loss_weights_dict):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    train_loss=0
    loss_dict={}
    for lbl,pred,loss_name in zip(labels,predictions,loss_funcs_dict.keys()):
         loss_func=loss_funcs_dict[loss_name]
         loss_weight=loss_weights_dict[loss_name]
         _loss,_=loss_func(lbl, pred)
         _loss = tf.reduce_mean(_loss)
         loss_dict[loss_name]=_loss
         train_loss+=loss_weight*_loss
  
  gradients = tape.gradient(train_loss, model.trainable_variables) 
    
  if _grad_clip != 0:
    for i in range(len(gradients)):
        if layer_no_dict[i] in list(grad_clip_layers.keys()):
             _clip_value=grad_clip_layers[layer_no_dict[i]]
             gradients[i]= tf.clip_by_value(gradients[i],-_clip_value,_clip_value) 
                
  curr_optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  loss_dict['loss']=train_loss
  return loss_dict


@tf.function
def get_grads(images, labels,_log_layers):
    grad_dict={}
    loss_names=list(loss_funcs_dict.keys())
    for i in range(4):
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss_func=loss_funcs_dict[loss_names[i]]
            _loss,_=loss_func(labels[i], predictions[i])
            _loss = tf.reduce_mean(_loss)
            train_params = [model.trainable_variables[layer_name_dict[_layer]] for _layer in _log_layers]
            gradients=tape.gradient(_loss, train_params)
        del tape
        
        if len(gradients)==1: 
            grad_dict[loss_names[i]] = gradients[0]
        else :
            _grads={}
            for j in range(len(gradients)):
                  _grads[_log_layers[j]]=gradients[j]
            grad_dict[loss_names[i]] = _grads

    return grad_dict

@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  val_loss=0
  loss_dict={}
  for lbl,pred,loss_name in zip(labels,predictions,loss_funcs_dict.keys()):
         loss_func=loss_funcs_dict[loss_name]
         loss_weight=loss_weights_dict[loss_name]
         _loss,_=loss_func(lbl, pred)
         _loss = tf.reduce_mean(_loss)
         loss_dict['val_'+loss_name+'_loss']=_loss
         val_loss+=loss_weight*_loss
  loss_dict['val_loss']=val_loss
  return loss_dict,predictions



def get_learning_rate(init_lr,epoch):
    lr_coeff=_lr_decay_epoch/np.log(_lr_decay_ratio)
    _epoch=-_lr_decay_enable*(epoch-1)
    epoch_lr=init_lr*np.exp(_epoch/lr_coeff)
    if epoch_lr <= _lr_min : epoch_lr= _lr_min
    #print('epoch: ',epoch,'lr:', epoch_lr)
    return epoch_lr


def get_disp_weight(wt_dict,epoch):
    init_wt=wt_dict['disp_4']
    wt_coeff=_disp_weight_decay_epoch/np.log(_disp_weight_decay_ratio)
    _epoch=-_disp_weight_decay_enable*(epoch-1)
    epoch_wt=init_wt*np.exp(_epoch/wt_coeff)
    if epoch_wt <= _disp_weight_min : epoch_wt= _disp_weight_min
    #print('epoch: ',epoch,'lr:', epoch_lr)
    wt_dict['disp_4']=np.float32(epoch_wt)
    return wt_dict

def get_flow_weight(wt_dict,epoch):
    init_wt=wt_dict['flow_4']
    wt_coeff=_flow_weight_decay_epoch/np.log(_flow_weight_decay_ratio)
    _epoch=-_flow_weight_decay_enable*(epoch-1)
    epoch_wt=init_wt*np.exp(_epoch/wt_coeff)
    if epoch_wt <= _flow_weight_min : epoch_wt= _flow_weight_min
    #print('epoch: ',epoch,'lr:', epoch_lr)
    wt_dict['flow_4']=np.float32(epoch_wt)
    return wt_dict

def get_l2_norm(r):
    r=tf.square(r)
    r=tf.reduce_sum(r)
    return r

def cosine_similarity(x,y):
    x=x.numpy()
    y=y.numpy()
    dot_prod=np.sum(x*y)
    mag_x=np.sqrt(np.sum(tf.square(x)))
    mag_y=np.sqrt(np.sum(tf.square(y)))
    _res=dot_prod/(mag_x*mag_y)
    #_res=tf.math.acos(_res)
    return _res

def get_cosine_similarity(_t,_r,_f,_d):
    _r=tf.reshape(_r,[-1])
    _t=tf.reshape(_t,[-1])
    _f=tf.reshape(_f,[-1])
    _d=tf.reshape(_d,[-1])

    _rf=cosine_similarity(_r,_f)#.numpy()
    _tf=cosine_similarity(_t,_f)#.numpy()
    _rt=cosine_similarity(_r,_t)#.numpy()
    _rd=cosine_similarity(_r,_d)#.numpy()
    _td=cosine_similarity(_t,_d)#.numpy()
    _fd=cosine_similarity(_f,_d)#.numpy()
    _res={}
    _res['rf_cos']=_rf
    _res['tf_cos']=_tf
    _res['rt_cos']=_rt
    _res['rd_cos']=_rd
    _res['td_cos']=_td
    _res['fd_cos']=_fd
    return _res

class loss_dict_obj(dict):
    def sum_update(self,c_dict):
        for key,val in c_dict.items():
                if key in list(self.keys()):
                    self[key]+=tf2np(c_dict[key])
                else:
                    self[key]=tf2np(c_dict[key])
                
    
    def ext_update(self,c_dict,_ext='_ext'):
        for key,val in c_dict.items():
                 self[key+_ext]=tf2np(val)
                
    def divide(self,div_val):
        for key,val in self.items():
            if type(div_val)==dict or type(div_val)==loss_dict_obj:
                self[key]/=div_val[key]
            else:
                self[key]/=div_val
                
    def _sum(self):
        total=0
        for val in self.values():
            total+=val
        return total
    
    def copy_keys(self,c_dict,keys):
        for key in keys:
            self[key]=c_dict[key]
            
    def apply(self,func):
        for key in self.keys():
            self[key]=tf2np(func(self[key]))
        
def tf2np(val):
    if hasattr(val,'numpy'):
        val=val.numpy()
    else:
        if val==None: val=0
    return val

class best_weight_saver():
    
        def __init__(self):
            self.min_val_loss=10e6
            self.min_val_mean_loss=10e6
            self.min_val_rot_loss=10e6
            self.min_val_trans_loss=10e6
            if not os.path.exists(log_dir+exp_name+'best_weights') : os.mkdir(log_dir+exp_name+'best_weights')
        def save_best_weight(self,_loss_dict,_loss='val_loss',_name='best_model'):
            if _loss=='val_loss':_min_loss=self.min_val_loss
            if _loss=='val_mean_loss':_min_loss=self.min_val_mean_loss
            if _loss=='val_rot_loss':_min_loss=self.min_val_rot_loss
            if _loss=='val_trans_loss':_min_loss=self.min_val_trans_loss
            if _loss_dict[_loss]<= _min_loss:
                _min_loss=_loss_dict[_loss]
                model.save(log_dir+exp_name+'best_weights//'+_name+'.h5')
                with open(log_dir+exp_name+'best_weights//'+_name+'_info.txt','w') as file:
                     file.write('epoch'+':'+str(epoch)+'\n')
                     for key,val in loss_dict.items():
                        file.write(str(key)+':'+str(val)+'\n')
            return _min_loss
            
        def save(self,_loss_dict):
            self.min_val_loss=self.save_best_weight(_loss_dict,_loss='val_loss',_name='best_model')
            self.min_val_mean_loss=self.save_best_weight(_loss_dict,_loss='val_mean_loss',_name='best_model_mean')
            self.min_val_rot_loss=self.save_best_weight(_loss_dict,_loss='val_rot_loss',_name='best_model_rot')
            self.min_val_trans_loss=self.save_best_weight(_loss_dict,_loss='val_trans_loss',_name='best_model_trans')
            return 0
        
def get_loss_rate(_loss):
    rel_loss_dict=loss_dict_obj()
    rel_loss_dict.copy_keys(_loss,['trans_loss','rot_loss','flow_4_loss','disp_4_loss'])
    #rel_loss_dict.divide(_init_loss)
    #total=rel_loss_dict._sum()
    #rel_loss_dict.divide(total)
    return rel_loss_dict  
        
if train == True :
    layer_no_dict={}
    layer_name_dict={}
    #{'grad_rate':_ratio_alpha}
    for i,wts in enumerate(model.trainable_variables):
            layer_no_dict[i]=wts.name
            layer_name_dict[wts.name]=i
    
    loss_log=loss_logger()
    weight_saver=best_weight_saver()
    loss_log.on_train_begin()

    loss_weights_dict={'trans':_init_loss_wts[0],
                       'rot':_init_loss_wts[1],
                       'flow_4':_init_loss_wts[2],
                       'disp_4':_init_loss_wts[3]}
    
    cosines=cosine_handler()
    slopes=slope_handler()
    
    _grad_var_dict={'rot':None,'flow_4':None,'disp_4':None}
    if debug==True:_total_epochs=start_epoch+25
    else:_total_epochs=start_epoch+epochs
    for epoch in range(start_epoch,_total_epochs):
        print("epoch : ",epoch,"/",_total_epochs-1)
        init_time=time.time()
        grads=loss_dict_obj()
        loss_dict=loss_dict_obj()
        rel_loss_dict=loss_dict_obj()
        loss_log.on_epoch_begin(epoch)
        
        loss_dict['epoch']=epoch
        loss_dict['curr_lr']=learning_rate
        if _lr_decay_enable == 1:                                               
               curr_lr=get_learning_rate(learning_rate,epoch)
               curr_optimizer.lr=copy(curr_lr)
               loss_dict['curr_lr']=curr_lr
                
        if _disp_weight_decay_enable==1:
               loss_weights_dict=get_disp_weight(loss_weights_dict,epoch)
        
        if _flow_weight_decay_enable==1:
               loss_weights_dict=get_flow_weight(loss_weights_dict,epoch)
                
        log_raw_grads={
            'trans':loss_dict_obj(),
            'rot':loss_dict_obj(),
            'flow_4':loss_dict_obj(),
            'disp_4':loss_dict_obj()
            }
        
       
        train_loss=loss_dict_obj()
        if debug==True:_tsteps=10
        else:_tsteps=len(train_data)//batch_size
        curr_grad_var_dict={'rot':None,'flow_4':None,'disp_4':None}
        for step in trange(_tsteps):
            
            images,labels=next(train_batch)
            if _mtl_get_variance_from == 'calc' and step%_mtl_grad_var_calc_every==0:
                #print('Calculating variance at step : ',step)
                curr_grad_var_dict['rot']=get_grad_var(images, labels,'rot')
                if _train_flow==True: curr_grad_var_dict['flow_4']=get_grad_var(images, labels,'flow_4')
                if _train_disp==True: curr_grad_var_dict['disp_4']=get_grad_var(images, labels,'disp_4')
                if _mtl_grad_var_alpha != 1.0:
                    for key in curr_grad_var_dict.keys():
                        if curr_grad_var_dict[key] ==None: continue
                        if curr_grad_var_dict[key] != None and _grad_var_dict[key] != None :
                            for _layer_ind in range(len(curr_grad_var_dict[key])):
                                _grad_var_dict[key][_layer_ind]=_mtl_grad_var_alpha*curr_grad_var_dict[key][_layer_ind]+(1-_mtl_grad_var_alpha)*_grad_var_dict[key][_layer_ind]
                        elif epoch==1 and step==0 :
                                _grad_var_dict[key]=curr_grad_var_dict[key]
                elif _mtl_grad_var_alpha == 1.0: _grad_var_dict = curr_grad_var_dict 
                if step%_mtl_update_grad_var==0:
                    if _mtl_div_with_rolling_mean==False: _updated_grad_var_dict=_grad_var_dict
                    elif _mtl_div_with_rolling_mean==True:
                        for key in curr_grad_var_dict.keys():
                            if curr_grad_var_dict[key] ==None: continue
                            if curr_grad_var_dict[key] != None and _grad_var_dict[key] != None :
                                for _layer_ind in range(len(curr_grad_var_dict[key])):
                                    curr_grad_var_dict[key][_layer_ind]=curr_grad_var_dict[key][_layer_ind]/(_grad_var_dict[key][_layer_ind]+10e-20)
                        _updated_grad_var_dict=curr_grad_var_dict

                    
            t_loss,calculated_grads,curr_cosines=train_step_mtl(images,labels,_updated_grad_var_dict)
            cosines.update(curr_cosines)
            curr_rel_loss_dict=get_loss_rate(t_loss)
            rel_loss_dict.sum_update(curr_rel_loss_dict)
            train_loss.sum_update(t_loss)
            
        del _updated_grad_var_dict,curr_grad_var_dict
        
        
        
        train_loss.divide(len(train_data)//batch_size)
        loss_dict.update(train_loss)
        
        
        if epoch==1 : 
            init_loss=copy(train_loss)
        #else:
        #    for loss_name in loss_funcs_dict.keys():
        #        if train_loss[loss_name+'_loss']>init_loss[loss_name+'_loss']:
        #            init_loss[loss_name+'_loss']=train_loss[loss_name+'_loss']
        rel_loss_dict.divide((len(train_data)//batch_size))
        rel_loss_dict.divide(init_loss)
        loss_dict.ext_update(rel_loss_dict,_ext='_rate')    
        loss_dict.ext_update(loss_weights_dict,_ext='_weight')
        
        
        slopes.update(_epoch=epoch,_curr_rel_loss=rel_loss_dict)
        loss_slope_dict=slopes.log(_epoch=epoch)
        loss_dict.update(loss_slope_dict)  
        
        cosine_log_dict=cosines.log(curr_cosines)
        loss_dict.update(cosine_log_dict)
        
        val_batch   = batch_generator(_val_data,_val_label)
        val_preds=np.zeros((1,no_of_pose))
        val_gts=np.zeros((1,no_of_pose))
        val_loss=loss_dict_obj()
        
        if debug==True:_vsteps=10
        else:_vsteps=len(val_data)//batch_size
        for step in trange(_vsteps):
            val_images,val_labels=next(val_batch)
            v_loss,val_pred=test_step(val_images,val_labels)
            val_loss.sum_update(v_loss)
            
            val_pred=np.concatenate(val_pred[0:2],axis=1)
            val_gt=np.concatenate(val_labels[0:2],axis=1)
            val_preds=np.concatenate((val_preds,val_pred),axis=0)
            val_gts=np.concatenate((val_gts,val_gt),axis=0)
            
        val_gts=unscl_label(val_gts)
        val_preds=unscl_label(val_preds)
        if not os.path.exists(log_dir+exp_name+'val_pred'): os.mkdir(log_dir+exp_name+'val_pred')
        with open(log_dir+exp_name+'val_pred\\'+'val_mat_'+str(epoch)+'.pkl','wb') as file:
                    pickle.dump([val_gts,val_preds],file)
            
            
        val_loss.divide(len(val_data)//batch_size)
        loss_dict.update(val_loss)        
        loss_dict['val_mean_loss']=(loss_dict['val_rot_loss']+loss_dict['val_trans_loss'])/2
        weight_saver.save(loss_dict)
        
        if debug==True:_grad_steps=10
        else:_grad_steps=_grad_data_len
            
        if  epoch%_grad_log_epoch==0 or epoch == 1: 
           for step in trange(_grad_steps):
                images,labels=next(grad_batch)
                curr_grads=get_grads(images,labels,grad_log_layers)
                for key in list(log_raw_grads.keys()):
                     log_raw_grads[key].sum_update(curr_grads[key])
           for key in list(log_raw_grads.keys()):
                 log_raw_grads[key].divide(_grad_data_len)
           for key in list(log_raw_grads.keys()):
                 grads[key]=log_raw_grads[key][grad_norm_layers[0]]
                  
        else :
            #for step in trange(_grad_data_len//10):
            for step in trange(_grad_steps):
                images,labels=next(grad_batch)
                curr_grads=get_grads(images,labels,grad_norm_layers)
                grads.sum_update(curr_grads)
            grads.divide(_grad_data_len)
        
        cosine_sim=get_cosine_similarity(grads['trans'],grads['rot'],grads['flow_4'],grads['disp_4'])  
        loss_dict.update(cosine_sim)
        grads.apply(get_l2_norm)
        loss_dict.ext_update(grads,_ext='_grad')
        loss_log.on_epoch_end(epoch,logs=loss_dict,grads=log_raw_grads,calculated_grads=calculated_grads,_curr_opt=curr_optimizer)
        for key,val in loss_dict.items():
            print(key,val)
        print('time required :', (time.time()-init_time)/60 ,' min')
        

In [ ]:
### END